# 1. Preparation

In [0]:
# Libs
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col
from pyspark.sql.types import DateType

#Datalake connection
blobAccessKey = dbutils.secrets.get(scope = "myscope", key = "accesskey")
spark.conf.set("fs.azure.account.key.datalakeetlproject.dfs.core.windows.net", 
               blobAccessKey) 
               
# Paths
path_bronze_profile = "abfss://bronze@datalakeetlproject.dfs.core.windows.net/profile/*.json"
path_bronze_historical = "abfss://bronze@datalakeetlproject.dfs.core.windows.net/historical/*.json"
path_silver_profile = "abfss://silver@datalakeetlproject.dfs.core.windows.net/profile/"
path_silver_historical = "abfss://silver@datalakeetlproject.dfs.core.windows.net/historical/"

# 2. Explore tables

## 2.1 Profile data

In [0]:
# Read data and show schema
df_bronze_profile = spark.read.json(path_bronze_profile)
df_bronze_profile.printSchema()

root
 |-- address: string (nullable = true)
 |-- beta: double (nullable = true)
 |-- ceo: string (nullable = true)
 |-- changes: double (nullable = true)
 |-- cik: string (nullable = true)
 |-- city: string (nullable = true)
 |-- companyName: string (nullable = true)
 |-- country: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- cusip: string (nullable = true)
 |-- dcf: double (nullable = true)
 |-- dcfDiff: double (nullable = true)
 |-- defaultImage: boolean (nullable = true)
 |-- description: string (nullable = true)
 |-- exchange: string (nullable = true)
 |-- exchangeShortName: string (nullable = true)
 |-- fullTimeEmployees: string (nullable = true)
 |-- image: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- ipoDate: string (nullable = true)
 |-- isActivelyTrading: boolean (nullable = true)
 |-- isAdr: boolean (nullable = true)
 |-- isEtf: boolean (nullable = true)
 |-- isFund: boolean (nullable = true)
 |-- isin: string (nullable = true)

In [0]:
# Show data
display(df_bronze_profile.limit(5))

address,beta,ceo,changes,cik,city,companyName,country,currency,cusip,dcf,dcfDiff,defaultImage,description,exchange,exchangeShortName,fullTimeEmployees,image,industry,ipoDate,isActivelyTrading,isAdr,isEtf,isFund,isin,lastDiv,mktCap,phone,price,range,sector,state,symbol,volAvg,website,zip
One Microsoft Way,1.055,Satya Nadella,11.53,0000789019,Redmond,Microsoft Corporation,US,USD,594918104,381.2641396727202,154.37586,false,"Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. The company operates in three segments: Productivity and Business Processes, Intelligent Cloud, and More Personal Computing. The Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, Microsoft Viva, and Skype for Business; Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for organizations and enterprise divisions. The Intelligent Cloud segment licenses SQL, Windows Servers, Visual Studio, System Center, and related Client Access Licenses; GitHub that provides a collaboration platform and code hosting service for developers; Nuance provides healthcare and enterprise AI solutions; and Azure, a cloud platform. It also offers enterprise support, Microsoft consulting, and nuance professional services to assist customers in developing, deploying, and managing Microsoft server and desktop solutions; and training and certification on Microsoft products. The More Personal Computing segment provides Windows original equipment manufacturer (OEM) licensing and other non-volume licensing of the Windows operating system; Windows Commercial, such as volume licensing of the Windows operating system, Windows cloud services, and other Windows commercial offerings; patent licensing; and Windows Internet of Things. It also offers Surface, PC accessories, PCs, tablets, gaming and entertainment consoles, and other devices; Gaming, including Xbox hardware, and Xbox content and services; video games and third-party video game royalties; and Search, including Bing and Microsoft advertising. The company sells its products through OEMs, distributors, and resellers; and directly through digital marketplaces, online stores, and retail stores. Microsoft Corporation was founded in 1975 and is headquartered in Redmond, Washington.",NASDAQ Global Select,NASDAQ,228000,https://images.financialmodelingprep.com/symbol/MSFT.png,Software - Infrastructure,1986-03-13,true,false,false,false,US5949181045,3.24,3981503178800,425 882 8080,535.64,344.79-555.45,Technology,WA,MSFT,19670220,https://www.microsoft.com,98052-6399
2788 San Tomas Expressway,2.145,Jen-Hsun Huang,6.28,0001045810,Santa Clara,NVIDIA Corporation,US,USD,67066G104,147.67472560535901,32.32527,false,"NVIDIA Corporation provides graphics, and compute and networking solutions in the United States, Taiwan, China, and internationally. The company's Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; vGPU software for cloud-based visual and virtual computing; automotive platforms for infotainment systems; and Omniverse software for building 3D designs and virtual worlds. Its Compute & Networking segment provides Data Center platforms and systems for AI, HPC, and accelerated computing; Mellanox networking and interconnect solutions; automotive AI Cockpit, autonomous driving development agreements, and autonomous vehicle solutions; cryptocurrency mining processors; Jetson for robotics and other embedded platforms; and NVIDIA AI Enterprise and other software. The company's products are used in gaming, professional visualization, datacenter, and automotive markets. NVIDIA Corporation sells its products to original equipment manufacturers, original device manufacturers, system buil

## 2.2 Hitorical data

In [0]:
# Read data and show schema
df_bronze_historical = spark.read.json(path_bronze_historical)
df_bronze_historical.printSchema()

root
 |-- historicalStockList: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- historical: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- adjClose: double (nullable = true)
 |    |    |    |    |-- change: double (nullable = true)
 |    |    |    |    |-- changeOverTime: double (nullable = true)
 |    |    |    |    |-- changePercent: double (nullable = true)
 |    |    |    |    |-- close: double (nullable = true)
 |    |    |    |    |-- date: string (nullable = true)
 |    |    |    |    |-- high: double (nullable = true)
 |    |    |    |    |-- label: string (nullable = true)
 |    |    |    |    |-- low: double (nullable = true)
 |    |    |    |    |-- open: double (nullable = true)
 |    |    |    |    |-- unadjustedVolume: long (nullable = true)
 |    |    |    |    |-- volume: long (nullable = true)
 |    |    |    |    |-- vwap: double (nullable = true)
 |    |    |-- symbol: s

In [0]:
# Expand to first level (historicalStockList)
df_level1 = df_bronze_historical.withColumn("stock", explode(col("historicalStockList")))

# Expand to sencond level (historical)
df_level2 = df_level1.withColumn("data", explode(col("stock.historical")))

# Select fields
df_bronze_historical = df_level2.select(
    col("stock.symbol"),
    col("data.date"),
    col("data.open"),
    col("data.high"),
    col("data.low"),
    col("data.close"),
    col("data.adjClose"),
    col("data.volume"),
    col("data.change"),
    col("data.changePercent"),
    col("data.vwap")
)

# Show data
display(df_bronze_historical.limit(20))

symbol,date,open,high,low,close,adjClose,volume,change,changePercent,vwap
MSFT,2025-08-04,528.27,538.25,528.13,535.64,535.64,25349004,7.37,1.4,532.5725
MSFT,2025-08-01,535.0,535.8,520.86,524.11,524.11,28977628,-10.89,-2.04,528.9425
MSFT,2025-07-31,555.23,555.45,531.9,533.5,533.5,51617326,-21.73,-3.91,544.02
MSFT,2025-07-30,515.17,515.95,509.44,513.24,513.24,26380434,-1.93,-0.37463,513.45
MSFT,2025-07-29,515.53,517.62,511.56,512.57,512.57,16469235,-2.96,-0.57417,514.32
MSFT,2025-07-28,514.08,515.0,510.12,512.5,512.5,14308027,-1.58,-0.30735,512.925
MSFT,2025-07-25,512.47,518.29,510.36,513.71,513.71,19125700,1.25,0.24197,513.7075
MSFT,2025-07-24,508.77,513.67,507.3,510.88,510.88,16107000,2.11,0.41473,510.155
MSFT,2025-07-23,506.75,506.79,500.7,505.87,505.87,16396600,-0.88,-0.17366,505.0275
MSFT,2025-07-22,510.97,511.2,505.27,505.27,505.27,13868644,-5.7,-1.12,508.1775


## 3. Data Transformation

## 3.1 Profile

In [0]:
# Copy 'profile' data from bronze layer
df_silver_profile = df_bronze_profile

# Select relevant fields and change column names
df_silver_profile_clean = df_silver_profile.select(
    col("symbol").alias("ticker"),
    col("companyName").alias("company_name"),
    col("sector"),
    col("industry"),
    col("country"),
    col("ceo").alias("ceo_name"),
    col("exchange"),
    col("mktCap").alias("market_cap"),
    col("price"),
    col("website"),
    col("description")
).dropDuplicates(["ticker"])



In [0]:
# display 'profile' data transformed
display(df_silver_profile_clean.limit(20))

ticker,company_name,sector,industry,country,ceo_name,exchange,market_cap,price,website,description
AAPL,Apple Inc.,Technology,Consumer Electronics,US,Timothy D. Cook,NASDAQ Global Select,3037194930000,203.35,https://www.apple.com,"Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as well as advertising services include third-party licensing arrangements and its own advertising platforms. In addition, the company offers various subscription-based services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening experience with on-demand radio stations; Apple News+, a subscription news and magazine service; Apple TV+, which offers exclusive original content; Apple Card, a co-branded credit card; and Apple Pay, a cashless payment service, as well as licenses its intellectual property. The company serves consumers, and small and mid-sized businesses; and the education, enterprise, and government markets. It distributes third-party applications for its products through the App Store. The company also sells its products through its retail and online stores, and direct sales force; and third-party cellular network carriers, wholesalers, retailers, and resellers. Apple Inc. was founded in 1976 and is headquartered in Cupertino, California."
ADBE,Adobe Inc.,Technology,Software - Infrastructure,US,Shantanu Narayen,NASDAQ Global Select,143740170000,338.85,https://www.adobe.com,"Adobe Inc. operates as a diversified software company worldwide. It operates through three segments: Digital Media, Digital Experience, and Publishing and Advertising. The Digital Media segment offers products, services, and solutions that enable individuals, teams, and enterprises to create, publish, and promote content; and Document Cloud, a unified cloud-based document services platform. Its flagship product is Creative Cloud, a subscription service that allows members to access its creative products. This segment serves content creators, workers, marketers, educators, enthusiasts, communicators, and consumers. The Digital Experience segment provides an integrated platform and set of applications and services that enable brands and businesses to create, manage, execute, measure, monetize, and optimize customer experiences from analytics to commerce. This segment serves marketers, advertisers, agencies, publishers, merchandisers, merchants, web analysts, data scientists, developers, and executives across the C-suite. The Publishing and Advertising segment offers products and services, such as e-learning solutions, technical document publishing, web conferencing, document and forms platform, web application development, and high-end printing, as well as Advertising Cloud offerings. The company offers its products and services directly to enterprise customers through its sales force and local field offices, as well as to end users through app stores and through its website at adobe.com. It also distributes products and services through a network of distributors, value-added resellers, systems integrators, software vendors and developers, retailers, and original equipment manufacturers. The company was formerly known as Adobe Systems Incorporated and changed its name to Adobe Inc. in October 2018. Adobe Inc. was founded in 1982 and is headquartered in San Jose, California."
ADP,"Automatic Data Processing, Inc.",Industrials,Staffing 

In [0]:
# Save into Silver layer
df_silver_profile_clean.write.mode("overwrite").format("delta").save(path_silver_profile)

## 3.2 Historical

In [0]:
df_bronze_historical.printSchema()


root
 |-- symbol: string (nullable = true)
 |-- date: string (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- adjClose: double (nullable = true)
 |-- volume: long (nullable = true)
 |-- change: double (nullable = true)
 |-- changePercent: double (nullable = true)
 |-- vwap: double (nullable = true)



In [0]:
# Copy 'historical' data from bronze layer
df_silver_historical = df_bronze_historical

# Expand arrays
df_silver_historical_level1 = df_silver_historical.withColumn("stock", explode(col("historicalStockList")))
df_silver_historical_level2 = df_silver_historical_level1.withColumn("data", explode(col("stock.historical")))

# Select relevant fields and change column names
df_silver_historical_clean = df_silver_historical.select(
    col("symbol").alias("ticker"),
    col("date").cast(DateType()).alias("date"),
    col("open"),
    col("high"),
    col("low"),
    col("close"),
    col("adjClose").alias("adj_close"),
    col("volume"),
    col("change"),
    col("changePercent").alias("change_percent"),
    col("vwap")
).dropDuplicates(["ticker", "date"])

In [0]:
# display 'historical' data transformed
display(df_silver_historical_clean.limit(20))

ticker,date,open,high,low,close,adj_close,volume,change,change_percent,vwap
MSFT,2024-12-03,429.84,432.47,427.74,431.2,429.55,18302000,1.36,0.3164,430.3125
BRK-B,2024-09-19,462.0,462.19,457.25,459.71,459.71,3891344,-2.29,-0.49567,460.2875
BRK-B,2024-09-09,463.97,466.0,459.26,459.61,459.61,4571041,-4.36,-0.93972,462.21
NVDA,2024-12-16,134.18,134.4,130.42,132.0,131.98,237951130,-2.18,-1.62,132.75
AAPL,2025-06-23,201.63,202.3,198.96,201.5,201.5,55814300,-0.125,-0.06447453,201.0975
AMZN,2024-12-31,222.97,223.23,218.94,219.39,219.39,24819700,-3.58,-1.61,221.1325
AMZN,2024-10-15,187.63,188.41,184.58,187.69,187.69,32178925,0.06,0.03197783,187.0775
MSFT,2025-03-25,393.92,396.36,392.64,395.16,394.44,15775000,1.25,0.31478,394.52
BRK-B,2025-03-25,527.18,533.29,525.88,528.87,528.87,3964800,1.69,0.32057,528.805
BRK-B,2025-03-12,497.6,498.32,488.75,495.94,495.94,3871100,-1.66,-0.3336,495.1525


In [0]:
# Save into Silver layer
df_silver_historical_clean.write.mode("overwrite").format("delta").save(path_silver_historical)